# Electricity Price Forecasting on the German day-ahead market

This notebook is the main interface to the associated EPF library. Various parameters can be adjusted within the library via the Config file. A more detailed description of the individual parameters can be found within the configuration classes. In the “exploratory_analysis” notebook, the raw data sets are analyzed exploratively and the results are displayed visually. Based on these results, features from the data sets were specifically selected for further use in the deep learning pipeline.

The forecasting pipeline is built to automatically perform data preprocessing including data cleaning, outlier removal and seasonal decomposition. Within the configuration feature engineering can be toggled on and of for each feature. Forecasting is done with two different models, that can be retrained and saved anytime. The models available are an LSTM and GRU. They all perform multi-step ahead, single shot forecasts. This means by default each forecast contains 24 time steps into the future, that are forecasted together in one computation.

In [1]:
from epf.pipeline import EpfPipeline

pipeline = EpfPipeline()

2025-05-12 16:58:22.065 | INFO     | epf.config:<module>:19 - PROJ_ROOT path is: C:\Users\valen\PycharmProjects\epf
2025-05-12 16:58:22.065 | INFO     | epf.config:create_dir:15 - DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data
2025-05-12 16:58:22.065 | INFO     | epf.config:create_dir:15 - RAW_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/raw
2025-05-12 16:58:22.065 | INFO     | epf.config:create_dir:15 - INTERIM_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/interim
2025-05-12 16:58:22.065 | INFO     | epf.config:create_dir:15 - PROCESSED_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/processed
2025-05-12 16:58:22.065 | INFO     | epf.config:create_dir:15 - TRAIN_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/processed/train_data
2025-05-12 16:58:22.065 | INFO     | epf.config:create_dir:15 - MODELS_DIR path is: C:/Users/valen/PycharmProjects/epf/models
2025-05-12 16:58:22.065 | INFO     | epf.config:create_dir:15 - PREDICTIONS_D

Pipeline initialized with:
FeatureConfig: 
Selected features: 
['DE-LU Prices', 'CH Prices', 'DK1 Prices', 'DK2 Prices', 'FR Prices', 'DE-LU Prices 7-Day Lag', 'DE-LU Prices 24-Hour Lag', 'DE-LU Prices 12-Hour Lag', 'DE-LU Prices 1-Hour Lag', 'DE Solar Generation', 'DE Wind Generation Offshore', 'DE Wind Generation Onshore', 'DE Gas Generation', 'DE Lignite Generation', 'DE Hard Coal Generation', 'DE Load', 'Month', 'Day of Week', 'Holiday']
WINDOW_LENGTH: 24
N_SIGMA: 3
METHOD: nearest

ModelConfig: 
TRAIN_SPLIT: 2023-09-30 22:00:00+00:00
VALIDATION_SPLIT: 2023-12-31 22:00:00+00:00
MAX_EPOCHS: 100
OUT_STEPS: 24
SEASONALITY_PERIOD: 24
INPUT_WIDTH_FACTOR: 1.25
MODEL_BUILDER: LSTM
USE_HIDDEN_LAYERS: True
NUM_FEATURES: 19
UNIT_MIN_VALUE: 32
UNIT_MAX_VALUE: 128
UNIT_STEP: 32
LR_MIN_VALUE: 0.001
LR_MAX_VALUE: 0.1
LR_STEP: 0.001
DROPOUT_RATE_MIN_VALUE: 0.2
DROPOUT_RATE_MAX_VALUE: 0.7
DROPOUT_RATE_STEP: 0.1
NUM_LAYERS_MIN: 1
NUM_LAYERS_MAX: 3
NUM_LAYERS_STEP: 1
MAX_TRIALS: 50
LABEL_COL: de_pri

In [2]:
model_name = 'lstm_all_features'

pipeline.train(model_name, overwrite=True, prep_data=True, use_tuned_hyperparams=False)
pipeline.evaluate(model_name)

Trial 50 Complete [00h 00m 22s]
mean_absolute_error: 0.2936801612377167

Best mean_absolute_error So Far: 0.27648964524269104
Total elapsed time: 00h 22m 55s


C:\Users\valen\.virtualenvs\bachelor_arbeit\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-05-12 17:24:22.447 | INFO     | epf.pipeline:_tune_hyperparameters:317 - Hyperparameter tuning took 00h:22m:57s.
2025-05-12 17:24:22.447 | SUCCESS  | epf.pipeline:train:424 - Successfully tuned hyperparameters for lstm_all_features
2025-05-12 17:24:22.447 | INFO     | epf.pipeline:train:431 - Training lstm_all_features on the following Features:['DE-LU Prices', 'CH Prices', 'DK1 Prices', 'DK2 Prices', 'FR Prices', 'DE-LU Prices 7-Day Lag', 'DE-LU Prices 24-Hour Lag', 'DE-LU Prices 12-Hour Lag', 'DE-LU Prices 1-Hour Lag', 'DE Solar Generation', 'DE Wind Generation Offshore', 'DE Wind Generation Onshore', 'DE Gas Generation', 'DE Lignite Generation', 'DE Hard Coal Generation', 'DE Load', 'Month

Epoch 1/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - loss: 0.2631 - mean_absolute_error: 0.2631 - mean_absolute_percentage_error: 216.7520 - root_mean_squared_error: 0.3654 - val_loss: 0.7108 - val_mean_absolute_error: 0.7108 - val_mean_absolute_percentage_error: 241.3448 - val_root_mean_squared_error: 0.9616
Epoch 2/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.2456 - mean_absolute_error: 0.2456 - mean_absolute_percentage_error: 216.3900 - root_mean_squared_error: 0.3418 - val_loss: 0.7019 - val_mean_absolute_error: 0.7019 - val_mean_absolute_percentage_error: 202.7307 - val_root_mean_squared_error: 0.9480
Epoch 3/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.2346 - mean_absolute_error: 0.2346 - mean_absolute_percentage_error: 205.9405 - root_mean_squared_error: 0.3279 - val_loss: 0.7219 - val_mean_absolute_error: 0.7219 - val_mean_absolute_percentage_error: 240.7725 - val_root_mean_squared_error: 0.9736
Epoch 4/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - lo

2025-05-12 17:24:42.658 | SUCCESS  | epf.pipeline:train:437 - Successfully trained lstm_all_features. Now saving...
2025-05-12 17:24:42.660 | INFO     | epf.pipeline:train:442 - Training took 00h:26m:18s.
2025-05-12 17:24:42.700 | SUCCESS  | epf.pipeline:train:466 - Successfully saved lstm_all_features to C:/Users/valen/PycharmProjects/epf/models/lstm_all_features.pkl
2025-05-12 17:24:42.700 | INFO     | epf.pipeline:evaluate:478 - Loading trained model from C:\Users\valen\PycharmProjects\epf\models\lstm_all_features.pkl.


68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7017 - mean_absolute_error: 0.7017 - mean_absolute_percentage_error: 289.3776 - root_mean_squared_error: 0.9463


2025-05-12 17:24:46.218 | INFO     | epf.pipeline:evaluate:511 - Evaluation took 00h:00m:03s.


In [ ]:
import pickle as pkl

with open("../models/lstm_all_features.pkl", "rb") as f:
    model_obj = pkl.load(f)

window = model_obj['window']
model = model_obj['best_model']
model_name = model_obj['model_name']
test_df = model_obj['test_df']
plot_col = 'de_prices_hat_rm_seasonal'

window.plot(model, plot_col=plot_col)